# converting to gray-scale 
grayscale_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [1]:
# importing necessary packages
import cv2
import numpy as np
import mediapipe
import itertools

In [2]:
mp_drawing=mediapipe.solutions.drawing_utils
mp_drawing_style=mediapipe.solutions.drawing_styles

drawing_specs=mp_drawing.DrawingSpec(color=(0,0,0),thickness=1)

### Get Shades overlay area

In [3]:
def getSize(image, face_landmarks):
    # Retrieve the height and width of the image.
    image_height, image_width, _ = image.shape
    
    # Convert the indexes of the landmarks of the face part into a list.
    INDEXES_LIST = [130,263]

    # Initialize a list to store the landmarks of the face part.
    landmarks = []

    # Iterate over the indexes of the landmarks of the face part. 
    for INDEX in INDEXES_LIST:
        # Append the landmark into the list.
        landmarks.append([int(face_landmarks[0].landmark[INDEX].x * image_width), int(face_landmarks[0].landmark[INDEX].y * image_height)])

    # Calculate the width and height of the face part.
    _, _, width, height = cv2.boundingRect(np.array(landmarks))

    # # Convert the list of landmarks of the face part into a numpy array.
    landmarks = np.array(landmarks)

    return width,height,landmarks

In [4]:
def overlay(image, filter_image, face_landmarks): 
    annotated_image = image.copy()
    
    try:
        # Get the width and height of filter image.
        filter_img_height, filter_img_width,_  = filter_image.shape
 
        # Get the height of the face part on which we will overlay the filter image.
        _, face_part_height,landmarks = getSize(image, face_landmarks)
        
        # Specify the height to which the filter image is required to be resized.
        required_height = int(face_part_height*15)
        
        # Resize the filter image to the required height, while keeping the aspect ratio constant. 
        resized_filter_img = cv2.resize(filter_image, (int(filter_img_width//3),required_height))
        
        # Get the new width and height of filter image.
        filter_img_height, filter_img_width, _  = resized_filter_img.shape
 
        # Convert the image to grayscale and apply the threshold to get the mask image.
        _, filter_img_mask = cv2.threshold(cv2.cvtColor(resized_filter_img, cv2.COLOR_BGR2GRAY),25, 255, cv2.THRESH_BINARY_INV)
 
        # Calculate the center of the face part.
        center = landmarks.mean(axis=0).astype("int")
 
        # calcualte the position to place the glasses
        location = (int(center[0]-filter_img_width/2), int(center[1]-filter_img_height/2))
 
        # Retrieve the region of interest from the image where the filter image will be placed.
        ROI = image[location[1]: location[1] + filter_img_height,location[0]: location[0] + filter_img_width]
 
        # Perform Bitwise-AND operation. This will set the pixel values of the region where,
        # filter image will be placed to zero.
        resultant_image = cv2.bitwise_and(ROI, ROI, mask=filter_img_mask)
 
        # Add the resultant image and the resized filter image.
        # This will update the pixel values of the resultant image at the indexes where 
        # pixel values are zero, to the pixel values of the filter image.
        resultant_image = cv2.add(resultant_image, resized_filter_img)
 
        # Update the image's region of interest with resultant image.
        annotated_image[location[1]: location[1] + filter_img_height, location[0]: location[0] + filter_img_width] = resultant_image
            
    # Catch and handle the error(s).
    except Exception as e:
        pass
    
    return annotated_image

In [8]:
# capturing video frame
vdf=cv2.VideoCapture(0)
shades=cv2.imread('shades.png')

face_mesh_model=mediapipe.solutions.face_mesh

with face_mesh_model.FaceMesh(
    max_num_faces=1,
    refine_landmarks= True,
    min_detection_confidence=0.5,
    min_tracking_confidence =0.5
) as face_mesh:
    
    while vdf.isOpened():
        success,image=vdf.read()
    
        if not success:
            break

        results=face_mesh.process(image)
        new_image=overlay(image,shades, results.multi_face_landmarks)
            
        cv2.imshow("Camera",cv2.flip(new_image,1))

        if cv2.waitKey(10)== ord('x'):
            break

vdf.release()
cv2.destroyAllWindows()

In [9]:
import numpy as np

def is_face_tilted(face_landmarks):
    # Assuming face_landmarks is a dictionary or list containing coordinates of facial landmarks
    # For example, if using dlib's facial landmarks, face_landmarks could be a shape object.

    # Extract eye landmarks (you may need to adjust these indices based on the specific landmarks you have)
    left_eye = np.array(face_landmarks[36:42])  # left eye landmarks
    right_eye = np.array(face_landmarks[42:48])  # right eye landmarks

    # Calculate the angle between the eyes
    angle = calculate_angle_between_eyes(left_eye, right_eye)

    # Define a threshold to determine if the face is tilted
    tilt_threshold = 10  # Adjust this threshold based on your requirements

    # Check if the angle is beyond the threshold
    return abs(angle) > tilt_threshold

def calculate_angle_between_eyes(left_eye, right_eye):
    # Calculate the angle between the line connecting eye landmarks
    # This function assumes the eye landmarks are ordered (x, y)

    # Calculate vectors representing the lines connecting the eyes
    vector_left = left_eye[-1] - left_eye[0]
    vector_right = right_eye[-1] - right_eye[0]

    # Calculate the cosine of the angle between the vectors
    dot_product = np.dot(vector_left, vector_right)
    magnitude_left = np.linalg.norm(vector_left)
    magnitude_right = np.linalg.norm(vector_right)

    # Ensure denominator is not zero
    if magnitude_left * magnitude_right == 0:
        return 0

    cosine_angle = dot_product / (magnitude_left * magnitude_right)

    # Calculate the angle in degrees
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

    return angle

# PYTHON UI 


In [ ]:
# No need to run
pip install wxpython

In [7]:
pip install -U -f https://extras.wxpython.org/wxPython4/extras/linux/gtk3/ubuntu-18.04/ wxPython

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://extras.wxpython.org/wxPython4/extras/linux/gtk3/ubuntu-18.04/
Note: you may need to restart the kernel to use updated packages.


In [9]:
# run
import wx

app = wx.App()
frame = wx.Frame(parent=None, title='Face Filter')
frame.Show()
app.MainLoop()

0